<a href="https://colab.research.google.com/github/HarshaVardhanLanka/OpenDeepResearchAgent/blob/main/Day_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install tavily-python

In [ ]:
pip install python-dotenv

In [ ]:
# Import necessary libraries
from tavily import TavilyClient
from google.colab import userdata  # For secure secrets

# Load the API key securely from Colab secrets
api_key = userdata.get('TAVILY')  # If not using secrets, use: api_key = "your-api-key-here" (insecure for sharing)

# Initialize the Tavily client
client = TavilyClient(api_key=api_key)

print("Tavily client initialized successfully!")

Tavily client initialized successfully!


In [ ]:
# Perform a simple search
response = client.search("What is the current weather in New York?", max_results=4)

# Print the results
print(response)

{'query': 'What is the current weather in New York?', 'follow_up_questions': None, 'answer': None, 'images': [], 'results': [{'title': 'Weather in New York', 'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'New York', 'region': 'New York', 'country': 'United States of America', 'lat': 40.7142, 'lon': -74.0064, 'tz_id': 'America/New_York', 'localtime_epoch': 1764072625, 'localtime': '2025-11-25 07:10'}, 'current': {'last_updated_epoch': 1764072000, 'last_updated': '2025-11-25 07:00', 'temp_c': 6.1, 'temp_f': 43.0, 'is_day': 1, 'condition': {'text': 'Partly cloudy', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 4.9, 'wind_kph': 7.9, 'wind_degree': 208, 'wind_dir': 'SSW', 'pressure_mb': 1024.0, 'pressure_in': 30.25, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 60, 'cloud': 75, 'feelslike_c': 4.5, 'feelslike_f': 40.0, 'windchill_c': 2.0, 'windchill_f': 35.6, 'heatindex_c': 3.8, 'heatindex_f': 38.8, 'dewpoint_c': 0.6, 'dewpoi

In [ ]:
!pip install -U langchain langchain-core langchain-google-genai langchain-community tavily-python google-generativeai pydantic

  Using cached langchain_google_genai-3.2.0-py3-none-any.whl.metadata (2.7 kB)
  Using cached google_generativeai-0.8.5-py3-none-any.whl.metadata (3.9 kB)
  Using cached filetype-1.2.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached google_ai_generativelanguage-0.9.0-py3-none-any.whl.metadata (10 kB)
INFO: pip is looking at multiple versions of google-generativeai to determine which version is compatible with other requirements. This could take a while.
  Using cached google_generativeai-0.8.4-py3-none-any.whl.metadata (4.2 kB)
  Using cached google_generativeai-0.8.3-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.2-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.8.0-py3-none-any.whl.metadata (3.9 kB)
  Using cached google_generativeai-0.7.2-py3-none-any.whl.metadata (4.0 kB)
  Using cached google_generativeai-0.7.1-py3-none-any.whl.metadata (3.9 kB)
INFO

In [ ]:
import os
import google.generativeai as genai
from tavily import TavilyClient
from google.colab import userdata

# 1. Setup API Keys
try:
    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
    TAVILY_API_KEY = userdata.get('TAVILY')
except:
    print("Error: Please set your API keys in the Colab Secrets tab (Key icon on the left).")

# 2. Configure Gemini
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel('gemini-flash-latest') # Using Flash for speed/efficiency

# 3. Configure Tavily
tavily = TavilyClient(api_key=TAVILY_API_KEY)

print("Setup complete! Ready to research.")

In [ ]:
def generate_sub_questions(topic):
    """Step 1: Decompose the topic into sub-questions using Gemini"""
    print(f"🤔 Generating sub-questions for: {topic}...")

    prompt = f"""
    You are a research assistant. Break down the following topic into 3 distinct,
    searchable sub-questions to gather comprehensive information.

    Topic: {topic}

    Output format: Return ONLY the 3 questions separated by newlines.
    Do not number them. Do not add introductory text.
    """

    response = model.generate_content(prompt)
    # Clean up the text to get a list of questions
    questions = [q.strip() for q in response.text.strip().split('\n') if q.strip()]

    return questions[:3] # Ensure we only return 3

def perform_research(questions):
    """Step 2: Search Tavily for each question"""
    print(f"🔎 Searching the web for answers...")
    research_data = []

    for q in questions:
        print(f"   - Searching: {q}")
        # usage of tavily search
        response = tavily.search(query=q, search_depth="basic", max_results=1)

        if response['results']:
            content = response['results'][0]['content']
            research_data.append(f"Question: {q}\nFound Info: {content}")
        else:
            research_data.append(f"Question: {q}\nFound Info: No relevant info found.")

    return "\n\n".join(research_data)

def synthesize_report(topic, research_data):
    """Step 3: Summarize everything into a final paragraph"""
    print(f"📝 Synthesizing final report...")

    prompt = f"""
    You are a professional researcher. Based ONLY on the provided research data below,
    write a concise, high-quality paragraph summarizing the findings regarding the topic: "{topic}".

    Research Data:
    {research_data}

    Final Summary:
    """

    response = model.generate_content(prompt)
    return response.text

In [ ]:
# --- Main Workflow ---

# 1. Take User Input
user_topic = input("Enter a research topic (e.g., 'The future of solid state batteries'): ")

if user_topic:
    print("-" * 50)

    # 2. Plan (Generate Questions)
    sub_questions = generate_sub_questions(user_topic)
    print(f"Generated Questions: {sub_questions}")
    print("-" * 50)

    # 3. Act (Search)
    raw_data = perform_research(sub_questions)
    # Optional: Print raw data to see what the agent found
    # print(f"Raw Data:\n{raw_data}")
    print("-" * 50)

    # 4. Synthesize (Final Output)
    final_summary = synthesize_report(user_topic, raw_data)

    print("\n🎯 FINAL RESEARCH SUMMARY:\n")
    print(final_summary)
else:
    print("Please enter a valid topic.")

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-2.5-pro-preview-03-25
models/gemini-2.5-flash
models/gemini-2.5-pro-preview-05-06
models/gemini-2.5-pro-preview-06-05
models/gemini-2.5-pro
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/learnlm-2.0-flash-experimental
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash